In [1]:
# !wget https://huggingface.co/datasets/syafie-nzm/crawl-jurnaldbp/resolve/main/jurnaldbp.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/mjpharm.org.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/myjgeosc.com.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/myjsustainagri.com.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/akademisains.gov.my.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/crossref-pdf.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/Kamus_Dewan_Bahasa_Edisi_Keempat_pdf.pdf
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/melayu-pdf.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/majcafe.com.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/myjms.mohe.gov.my.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/newera.edu.my.jsonl

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [2]:
from glob import glob
from tqdm import tqdm
import json
import os
import numpy as np
from transformers import AutoTokenizer
from streaming import MDSWriter
from tqdm import tqdm
import msgspec
import json
import re

http_errors = [
        "400 Bad Request", "401 Unauthorized", "402 Payment Required", "403 Forbidden", "404 Not Found",
        "405 Method Not Allowed", "406 Not Acceptable", "407 Proxy Authentication Required", "408 Request Timeout",
        "409 Conflict", "410 Gone", "411 Length Required", "412 Precondition Failed", "413 Payload Too Large",
        "414 URI Too Long", "415 Unsupported Media Type", "416 Range Not Satisfiable", "417 Expectation Failed",
        "418 I'm a teapot", "421 Misdirected Request", "422 Unprocessable Entity", "423 Locked", "424 Failed Dependency",
        "425 Too Early", "426 Upgrade Required", "428 Precondition Required", "429 Too Many Requests",
        "431 Request Header Fields Too Large", "451 Unavailable For Legal Reasons", "500 Internal Server Error",
        "501 Not Implemented", "502 Bad Gateway", "503 Service Unavailable", "504 Gateway Timeout",
        "505 HTTP Version Not Supported", "506 Variant Also Negotiates", "507 Insufficient Storage",
        "508 Loop Detected", "510 Not Extended", "511 Network Authentication Required"
    ]

rejected = [
    'Internal Server Error',
    '__NOEDITSECTION__',
    'enter your username and password',
    'forgotten your password',
    'cookies enabled',
    'enable JavaScript in your browser.',
    'The page cannot be displayed',
    'site or edit the error_page',
    'Request unsuccessful',
]

rejected.extend(http_errors)

def replace_multiple(input_string, pattern =r"\s{6,}", replace = '   '):
    return re.sub(pattern, replace, input_string)

def replace(string):
    string = replace_multiple(string.replace('…', '.'))
    string = replace_multiple(string, pattern = r"\.{6,}", replace = '...')
    return string

def reject(string):
    if any([r in string for r in rejected]):
        return True
    return False

In [3]:
from streaming.base.format.mds.encodings import Encoding, _encodings

class UInt16(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint16)

_encodings['uint16'] = UInt16

In [4]:
files = [
    'jurnaldbp.jsonl',
    'mjpharm.org.jsonl',
    'myjgeosc.com.jsonl',
    'myjsustainagri.com.jsonl',
    'akademisains.gov.my.jsonl',
    'crossref-pdf.jsonl',
    'Kamus_Dewan_Bahasa_Edisi_Keempat_pdf.pdf',
    'melayu-pdf.jsonl',
    'majcafe.com.jsonl',
    'myjms.mohe.gov.my.jsonl',
    'newera.edu.my.jsonl'
]

In [5]:
columns = {
    'input_ids': 'uint16',
}
compression = 'zstd'
hashes = 'sha1', 'xxh64'

In [6]:
def partition(text, size = 500):
    splitted = text.split()
    return [' '.join(splitted[i: i + size]) for i in range(0, len(splitted), size)]

In [7]:
with open('extra.jsonl', 'w') as fopen_l:
    for f in files:
        with open(f, encoding = "ISO-8859-1") as fopen:
            for l in tqdm(fopen):
                try:
                    l = msgspec.json.decode(l)
                    if reject(l):
                        continue

                    l = replace(l.strip())
                    if len(l) < 3:
                        continue
                        
                    data = '<|startoftext|>' + l + '<|endoftext|>'
                    partitioned = partition(data)
                    for p in partitioned:
                        data = {
                            'text': p,
                        }
                        fopen_l.write(f'{json.dumps(data)}\n')
                        fopen_l.flush()
                except Exception as e:
                    print(e)
                    pass

930it [00:02, 371.82it/s]
90it [00:00, 172.87it/s]
100it [00:00, 393.61it/s]
119it [00:00, 434.32it/s]
811it [00:01, 412.04it/s]
62149it [04:48, 215.22it/s]
53910it [00:00, 268232.21it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

133860it [00:00, 228015.97it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

230240it [00:00, 270994.82it/s]


JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is malformed: invalid number (byte 1)
JSON is mal

1506it [00:04, 309.39it/s]
490it [00:01, 424.71it/s]
16419it [00:33, 496.40it/s]
87it [00:00, 688.09it/s]


In [8]:
!wc -l extra.jsonl

1286178 extra.jsonl


In [9]:
split_by = 100000

In [10]:
!mkdir partitions-extra

In [11]:
index = 0
count = 0
a = open(f'partitions-extra/combined-lm-{index}.jsonl', 'w')

with open('extra.jsonl') as fopen:
    for l in tqdm(fopen):
        a.write(l)
        a.flush()
        count += 1
        if count >= split_by:
            a.close()
            index += 1
            count = 0
            a = open(f'partitions-extra/combined-lm-{index}.jsonl', 'w')
            
a.close()

1286178it [00:13, 98102.25it/s] 


In [12]:
block_size = 4096

def read_dataset(train_file, block_size = block_size):
    
    tokenizer = AutoTokenizer.from_pretrained(
    '01-ai/Yi-6B',
    )
    tokenizer.add_bos_token = False
    tokenizer.add_eos_token = False
    text_column_name = 'text'
    temp = []
    with open(train_file) as fopen:
        for l in fopen:
            l = msgspec.json.decode(l)
            tokenized = tokenizer(l['text'])['input_ids']
            temp.extend(tokenized)
            while len(temp) >= block_size:
                block = temp[:block_size]
                temp = temp[block_size:]
                if len(block) == block_size:
                    yield np.array(block).astype(np.uint16)

In [13]:
files = sorted(glob('partitions-extra/combined-lm-*.jsonl'), key = lambda x: int(x.split('-')[-1].replace('.jsonl', '')))
files

['partitions-extra/combined-lm-0.jsonl',
 'partitions-extra/combined-lm-1.jsonl',
 'partitions-extra/combined-lm-2.jsonl',
 'partitions-extra/combined-lm-3.jsonl',
 'partitions-extra/combined-lm-4.jsonl',
 'partitions-extra/combined-lm-5.jsonl',
 'partitions-extra/combined-lm-6.jsonl',
 'partitions-extra/combined-lm-7.jsonl',
 'partitions-extra/combined-lm-8.jsonl',
 'partitions-extra/combined-lm-9.jsonl',
 'partitions-extra/combined-lm-10.jsonl',
 'partitions-extra/combined-lm-11.jsonl',
 'partitions-extra/combined-lm-12.jsonl']

In [14]:
next(read_dataset(files[1]))

array([1197, 1827,  593, ..., 6257,   98, 1289], dtype=uint16)

In [15]:
!rm -rf tokenized_extra
!mkdir tokenized_extra

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
def loop(files):
    files, index = files
    out_root = f'tokenized_extra/tokenized-{index}'
    os.system(f'rm -rf {out_root}')
    with MDSWriter(out=out_root, columns=columns, compression=compression, hashes=hashes) as out:
        for f in files:
            for block in tqdm(read_dataset(train_file = f)):
                sample = {
                    'input_ids': block
                }
                out.write(sample)

In [17]:
import mp
mp.multiprocessing(files, loop, cores = 10, returned = False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [19]:
from streaming import StreamingDataset

total = 0
for f in glob('tokenized_extra/tokenized-*'):
    dataset = StreamingDataset(local = f)
    total += len(dataset)
total * block_size

1258356736